This notebook will analyze all other cases as they only pertain to specific convective modes. Have fun with it!

In [ ]:
#Supress Warnings
import warnings
warnings.filterwarnings("ignore")

from utils import integral_ts

import numpy as np
import pickle
import cmocean
from datetime import datetime
import scipy.signal as sig
from scipy.ndimage import gaussian_filter
import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator, FormatStrFormatter,AutoMinorLocator
import matplotlib.dates as mdates

#Import the case dictionaries
f = open('clamps_cases_test.pckl', 'rb')
lin,dis,mix = pickle.load(f)
f.close()

l_height = 2500 #m
l_height_barb = 1500 #m 2500

plt.rcParams.update({'font.size':30,'font.family':'Arial'})

In [ ]:
#Print out all case dates to pick the index
data = lin
print('Linear')
[print('{}: {}'.format(k,data[k]['date'])) for k in range(len(data))]
print()

data = dis
print('Discrete')
[print('{}: {}'.format(k,data[k]['date'])) for k in range(len(data))]
print()

data = mix
print('Mixed')
[print('{}: {}'.format(k,data[k]['date'])) for k in range(len(data))]
print()

In [ ]:
#Pick a case here by choosing an index and a dictionary list 
data = dis[1]

#Get the height
t_height = data['t_height']
l_height = data['l_height']

#Get the time
t_dec,l_dec = data['t_dec'],data['l_dec']
t_utc,l_utc = data['t_utc'],data['l_utc']

#Get all other variables
temp,th,pres,dew,qv = data['temp'],data['theta'],data['pres'],data['dew'],data['qv']
cin,cape = data['sbcin'],data['sbcape2d']
u,v,w,wspd,sh01,sh03,sh13 = data['u'],data['v'],data['w'],data['wspd'],data['mag01'],data['mag03'],data['mag13']


In [ ]:
#######################################################
#Plot the vertical velocity from the vertical stare
#######################################################

hidx = np.where(l_height>=2000)[0][0]
height_plot = l_height[:hidx]

#Use this to append to the dataframe 
timestamp=l_utc
    
#Try to make the data gaps all nans so they do not get smoothed over
df = pd.DataFrame(w[:,:hidx])
df.set_index(np.array(timestamp),inplace=True)
df = df.resample('5T').first().fillna(value=np.nan)
w_plot1 = df.values.T

df.reset_index(inplace=True)
timestamp = df['index'].values

#Format the colorbar
levs = np.linspace(-1,1,256,endpoint=True)
norm = mpl.colors.BoundaryNorm(levs,256)

#Plot the data
plt.figure(figsize=(13,7))
ax = plt.subplot(1,1,1)
ax.set_title('Vertical Velocity: {}'.format(datetime.strptime(str(data['date']),'%Y-%m-%d %H:%M:%S').strftime('%Y %B %d')),loc='left')

X,Y = np.meshgrid(timestamp,height_plot)
color_map = plt.cm.get_cmap('seismic')
plot = ax.contourf(X,Y,w_plot1,cmap=color_map,norm=norm,levels=levs,extend='max')
plot.cmap.set_under('grey') #Grey undertones for bad data

ax.axvline(x=data['passage_time'],linestyle='dashed',color='k',label='Storm Passage',linewidth=3)

#Format axes
#ax.set_xlabel('Time [UTC]')
ax.xaxis.set_major_locator(mdates.MinuteLocator(interval=60))
ax.xaxis.set_minor_locator(mdates.MinuteLocator(interval=30))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
plt.setp(ax.xaxis.get_majorticklabels(), rotation=45)

ax.set_ylabel('Height [m]')
ax.yaxis.set_major_locator(MultipleLocator(400))
ax.yaxis.set_major_formatter(FormatStrFormatter('%d'))
ax.yaxis.set_minor_locator(MultipleLocator(200))

ax.set_ylim((0,2700)) 
ax.set_xlim(timestamp[0],timestamp[-1])

#Legend
#ax.legend(framealpha=0.0,loc='upper left')

plt.tight_layout()
#plt.savefig('figures/20170310/vert_vel_{}.jpg'.format(datetime.strptime(str(data['date']),'%Y-%m-%d %H:%M:%S').strftime('%Y%m%d')),dpi=300,bbox_inches='tight')
plt.show()
plt.close()


In [ ]:
#Colorbar stuff
fig = plt.figure(figsize=(10,10))

ticks = np.linspace(-1,1,5,endpoint=True)
rad_cax = fig.add_axes([0.15, 0.05, 0.03, 0.45]) #zonal, vertical, width, length
rad_cb = plt.colorbar(plot,ticks=ticks,cax=rad_cax,norm=norm,orientation='vertical')
rad_cax.yaxis.set_ticks_position('right')
#rad_cax.yaxis.set_label_position('right')
#rad_cb.set_label(label='m $s^{-1}$')

#plt.savefig('figures/20170310/vert_vel_colorbar.png',dpi=300,bbox_inches='tight')

plt.show()

In [ ]:
#######################################################
#Plot the horizontal wind speed with wind barbs
#######################################################

hidx = np.where(l_height>=2000)[0][0]
height_plot = l_height[:hidx]

#Use this to append to the dataframe 
timestamp=l_utc
    
#Try to make the data gaps all nans so they do not get smoothed over
df = pd.DataFrame(wspd[:,:hidx])
df.set_index(np.array(timestamp),inplace=True)
df = df.resample('5T').first().fillna(value=np.nan)
wspd_plot1 = df.values.T

df.reset_index(inplace=True)
timestamp = df['index'].values

#Get the data to plot and filter
u_plot = u[:,:hidx]
v_plot = v[:,:hidx]

wspd_plot1[wspd_plot1<=0] = np.nan
wspd_plot1[wspd_plot1==2.0000002] = np.nan

u[u==-9999] = np.nan
v[v==-9999] = np.nan


#Format the colorbar
levs = np.linspace(0,30,256,endpoint=True)
norm = mpl.colors.BoundaryNorm(levs,256)

#Plot the data
plt.figure(figsize=(13,7))
ax = plt.subplot(1,1,1)
ax.set_title('Horizontal Velocity: {}'.format(datetime.strptime(str(data['date']),'%Y-%m-%d %H:%M:%S').strftime('%Y %B %d')),loc='left')

X,Y = np.meshgrid(timestamp,height_plot)
color_map = plt.cm.get_cmap('gist_stern')
plot = ax.contourf(X,Y,wspd_plot1,cmap=color_map.reversed(),norm=norm,levels=levs,extend='max')
plot.cmap.set_under('grey') #Grey undertones for bad data

ax.axvline(x=data['passage_time'],linestyle='dashed',color='k',label='Storm Passage',linewidth=3)

#Plot the wind barbs
time_incre = 12 #10
height_incre = 8 #10

#Use this to append to the dataframe 
timestamp=l_utc

x,y = np.meshgrid(timestamp[::time_incre],height_plot[::height_incre])
u_plot2 = u_plot[::time_incre,::height_incre] * 1.943844 #knots
v_plot2 = v_plot[::time_incre,::height_incre] * 1.943844 #knots
barb = ax.barbs(x,y,u_plot2.transpose(),v_plot2.transpose(),length=7,linewidth=0.7)

#Format axes
ax.set_xlabel('Time [UTC]')
ax.xaxis.set_major_locator(mdates.MinuteLocator(interval=60))
ax.xaxis.set_minor_locator(mdates.MinuteLocator(interval=30))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
plt.setp(ax.xaxis.get_majorticklabels(), rotation=45)

ax.set_ylabel('Height [m]')
ax.yaxis.set_major_locator(MultipleLocator(400))
ax.yaxis.set_major_formatter(FormatStrFormatter('%d'))
ax.yaxis.set_minor_locator(MultipleLocator(200))

ax.set_ylim((0,2400)) 
ax.set_xlim(timestamp[0],timestamp[-1])

#Legend
#ax.legend(framealpha=0.0,loc='upper left')

plt.tight_layout()
#plt.savefig('figures/20170310/horiz_vel_{}.jpg'.format(datetime.strptime(str(data['date']),'%Y-%m-%d %H:%M:%S').strftime('%Y%m%d')),dpi=300,bbox_inches='tight')
plt.show()
plt.close()


In [ ]:
#Colorbar stuff
fig = plt.figure(figsize=(10,10))

ticks = np.arange(0,40,6)
rad_cax = fig.add_axes([0.15, 0.05, 0.03, 0.45]) #zonal, vertical, width, length
rad_cb = plt.colorbar(plot,ticks=ticks,cax=rad_cax,norm=norm,orientation='vertical')
rad_cax.yaxis.set_ticks_position('right')
#rad_cax.yaxis.set_label_position('right')
#rad_cb.set_label(label='m $s^{-1}$')

#plt.savefig('figures/20170310/horiz_vel_colorbar.png',dpi=300,bbox_inches='tight')

plt.show()

In [ ]:
#######################################################
#Plot water vapor mixing ratio
#######################################################

hidx = np.where(t_height>=2500)[0][0]
height_plot = t_height[:hidx]

#Use this to append to the dataframe 
timestamp=t_utc
    
#Try to make the data gaps all nans so they do not get smoothed over
df = pd.DataFrame(qv[:,:hidx])
df.set_index(np.array(timestamp),inplace=True)
df = df.resample('10T').first().fillna(value=np.nan)
qv_plot1 = df.values.T

df.reset_index(inplace=True)
timestamp = df['index'].values

#Format the colorbar
levs = np.linspace(0,16,256,endpoint=True)
norm = mpl.colors.BoundaryNorm(levs,256)

#Plot the data
plt.figure(figsize=(13,7))
ax = plt.subplot(1,1,1)
ax.set_title('Water Vapor Mixing Ratio: {}'.format(datetime.strptime(str(data['date']),'%Y-%m-%d %H:%M:%S').strftime('%Y %B %d')),loc='left')

X,Y = np.meshgrid(timestamp,height_plot)
color_map = plt.cm.get_cmap('Greens')
plot = ax.contourf(X,Y,qv_plot1,cmap=color_map,norm=norm,levels=levs,extend='max')
contours = ax.contour(X,Y,qv_plot1,levels=[10],colors='white')
#ax.clabel(contours,fmt='%1.0f')

ax.axvline(x=data['passage_time'],linestyle='dashed',color='k',label='Storm Passage',linewidth=3)

#Format axes
#ax.set_xlabel('Time [UTC]')
ax.xaxis.set_major_locator(mdates.MinuteLocator(interval=60))
ax.xaxis.set_minor_locator(mdates.MinuteLocator(interval=30))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
plt.setp(ax.xaxis.get_majorticklabels(), rotation=45)

ax.set_ylabel('Height [m]')
ax.yaxis.set_major_locator(MultipleLocator(400))
ax.yaxis.set_major_formatter(FormatStrFormatter('%d'))
ax.yaxis.set_minor_locator(MultipleLocator(200))

ax.set_ylim((0,2400)) 
ax.set_xlim(timestamp[0],timestamp[-1])

#Legend
ax.legend(framealpha=0.0,loc='upper left')

plt.tight_layout()
#plt.savefig('figures/20170310/wvmr_{}.jpg'.format(datetime.strptime(str(data['date']),'%Y-%m-%d %H:%M:%S').strftime('%Y%m%d')),dpi=300,bbox_inches='tight')
plt.show()
plt.close()


In [ ]:
#Colorbar stuff
fig = plt.figure(figsize=(10,10))

ticks = np.arange(0,18,3)
rad_cax = fig.add_axes([0.15, 0.05, 0.03, 0.45]) #zonal, vertical, width, length
rad_cb = plt.colorbar(plot,ticks=ticks,cax=rad_cax,norm=norm,orientation='vertical')
rad_cax.yaxis.set_ticks_position('right')
#rad_cax.yaxis.set_label_position('right')
#rad_cb.set_label(label='g $kg^{-1}$')

#plt.savefig('figures/20170310/wvmr_colorbar.png',dpi=300,bbox_inches='tight')

plt.show()

In [ ]:
#######################################################
#Plot potential temperature
#######################################################

hidx = np.where(t_height>=2500)[0][0]
height_plot = t_height[:hidx]

#Use this to append to the dataframe 
timestamp=t_utc

#Try to make the data gaps all nans so they do not get smoothed over
df = pd.DataFrame(th[:,:hidx])
df.set_index(np.array(timestamp),inplace=True)
df = df.resample('10T').first().fillna(value=np.nan)
th_plot1 = df.values.T

#th_plot1[th_plot1<=280] = np.nan

df.reset_index(inplace=True)
timestamp = df['index'].values

#Format the colorbar
levs = np.linspace(280,305,100,endpoint=True)
norm = mpl.colors.BoundaryNorm(levs,256)

#Plot the data
plt.figure(figsize=(13,7))
ax = plt.subplot(1,1,1)
ax.set_title('Potential Temperature: {}'.format(datetime.strptime(str(data['date']),'%Y-%m-%d %H:%M:%S').strftime('%Y %B %d')),loc='left')

X,Y = np.meshgrid(timestamp,height_plot)
color_map = cmocean.cm.thermal
plot = ax.contourf(X,Y,th_plot1,cmap=color_map,norm=norm,levels=levs,extend='max')
contours = ax.contour(X,Y,th_plot1,levels=np.arange(295,305,3),colors='k')
ax.clabel(contours,fmt='%1.0f')

ax.axvline(x=data['passage_time'],linestyle='dashed',color='k',label='Storm Passage',linewidth=3)

#Format axes
#ax.set_xlabel('Time [UTC]')
ax.xaxis.set_major_locator(mdates.MinuteLocator(interval=60))
ax.xaxis.set_minor_locator(mdates.MinuteLocator(interval=30))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
plt.setp(ax.xaxis.get_majorticklabels(), rotation=45)

#ax.set_ylabel('Height [m]')
ax.yaxis.set_major_locator(MultipleLocator(400))
ax.yaxis.set_major_formatter(FormatStrFormatter('%d'))
ax.yaxis.set_minor_locator(MultipleLocator(200))

ax.set_ylim((0,2400)) 
ax.set_xlim(timestamp[0],timestamp[-1])

#Legend
#ax.legend(framealpha=0.0,loc='upper left')

plt.tight_layout()
#plt.savefig('figures/20170310/th_{}.jpg'.format(datetime.strptime(str(data['date']),'%Y-%m-%d %H:%M:%S').strftime('%Y%m%d')),dpi=300,bbox_inches='tight')
plt.show()
plt.close()


In [ ]:
#Colorbar stuff
fig = plt.figure(figsize=(10,10))

ticks = np.arange(275,310,5)
rad_cax = fig.add_axes([0.15, 0.05, 0.03, 0.45]) #zonal, vertical, width, length
rad_cb = plt.colorbar(plot,ticks=ticks,cax=rad_cax,norm=norm,orientation='vertical')
rad_cax.yaxis.set_ticks_position('right')
#rad_cax.yaxis.set_label_position('right')
#rad_cb.set_label(label=r'K')

#plt.savefig('figures/20170310/th_colorbar.png',dpi=300,bbox_inches='tight')

plt.show()

In [ ]:
#######################################################
#Plot the integrated CAPE
#######################################################

hidx = np.where(t_height>=3000)[0][0]
height_plot = t_height[:hidx]

#Use this to append to the dataframe 
timestamp=t_utc
    
#Try to make the data gaps all nans so they do not get smoothed over
df = pd.DataFrame(cape)
df.set_index(np.array(timestamp),inplace=True)
df = df.resample('10T').first().fillna(value=np.nan)
sbcape2d_plot1 = df.values.T

df.reset_index(inplace=True)
timestamp = df['index'].values

#Format the colorbar
levs = np.linspace(0,3500,256,endpoint=True)
norm = mpl.colors.BoundaryNorm(levs,256)

#Plot the data
plt.figure(figsize=(13,7))
ax = plt.subplot(1,1,1)
ax.set_title('Integrated CAPE: {}'.format(datetime.strptime(str(data['date']),'%Y-%m-%d %H:%M:%S').strftime('%Y %B %d')),loc='left')

X,Y = np.meshgrid(timestamp,height_plot)
color_map = plt.get_cmap('turbo')
plot = ax.contourf(X,Y,sbcape2d_plot1,cmap=color_map,norm=norm,levels=levs,extend='max')

ax.axvline(x=data['passage_time'],linestyle='dashed',color='k',label='Storm Passage',linewidth=3)

#Format axes
ax.set_xlabel('Time [UTC]')
ax.xaxis.set_major_locator(mdates.MinuteLocator(interval=60))
ax.xaxis.set_minor_locator(mdates.MinuteLocator(interval=30))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
plt.setp(ax.xaxis.get_majorticklabels(), rotation=45)

ax.set_ylabel('Height [m]')
ax.yaxis.set_major_locator(MultipleLocator(400))
ax.yaxis.set_major_formatter(FormatStrFormatter('%d'))
ax.yaxis.set_minor_locator(MultipleLocator(200))

ax.set_ylim((0,2400)) 
ax.set_xlim(timestamp[0],timestamp[-1])

plt.tight_layout()
#plt.savefig('figures/20170405/sbcape_2d_{}.jpg'.format(datetime.strptime(str(data['date']),'%Y-%m-%d %H:%M:%S').strftime('%Y%m%d')),dpi=300,bbox_inches='tight')
plt.show()
plt.close()


In [ ]:
#Colorbar stuff
fig = plt.figure(figsize=(10,10))

ticks = np.arange(0,5000,800)
rad_cax = fig.add_axes([0.15, 0.05, 0.03, 0.45]) #zonal, vertical, width, length
rad_cb = plt.colorbar(plot,ticks=ticks,cax=rad_cax,norm=norm,orientation='vertical')
rad_cax.yaxis.set_ticks_position('right')
rad_cax.yaxis.set_label_position('right')
#rad_cb.set_label(label=r'J $kg^{-1}$')

#plt.savefig('figures/20170310/sbcape2d_colorbar.png',dpi=300,bbox_inches='tight')

plt.show()

In [ ]:
#######################################################
#Plot time series of vertical shear
#######################################################

timestamp=l_utc
    
plt.figure(figsize=(12,5))
plt.title('0-1km Veritcal Wind Shear',loc='left')

ax = plt.subplot(1,1,1)
ax.plot(timestamp,sh01,color='blue',linewidth=2,linestyle='solid',label='0-1km Shear')
#ax.plot(timestamp,sh03,color='red',linewidth=2,linestyle='solid',label='0-3km Shear')
#ax.plot(timestamp,sh13,color='green',linewidth=2,linestyle='solid',label='1-3km Shear')

#Format the axes
ax.set_xlabel('Time [UTC]')
ax.xaxis.set_major_locator(mdates.MinuteLocator(interval=60))
ax.xaxis.set_minor_locator(mdates.MinuteLocator(interval=30))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
plt.setp(ax.xaxis.get_majorticklabels(), rotation=45)

ax.set_ylabel('m $s^{-1}$')
ax.yaxis.set_major_locator(MultipleLocator(5))
ax.yaxis.set_major_formatter(FormatStrFormatter('%d'))
ax.yaxis.set_minor_locator(MultipleLocator(2.5))

plt.show()
plt.close()

In [ ]:
#######################################################
#Compute the integral time scale of all features
#######################################################

plt.rcParams.update({'font.size':25})
plt.figure(figsize=(10,10))
plt.grid(alpha=0.6)
ax = plt.subplot(1,1,1)
ax.set_title('Time Scales: {}'.format(datetime.strptime(str(dis[0]['date']),'%Y-%m-%d %H:%M:%S').strftime('%Y %B %d')),loc='left')

h1 = np.where(t_height>=1500)[0][0]
h2 = np.where(l_height>=1500)[0][0]

#Plot qv
lag_data,efolding_time = integral_ts(qv,(t_dec - t_dec[0]),t_height[:h1])
ax.plot(lag_data,t_height[:h1],color='green',label='Mixing Ratio',linewidth=3,linestyle='solid')

#plot theta
lag_data,efolding_time = integral_ts(th*(1+(0.61*(qv/1000))),(t_dec - t_dec[0]),t_height[:h1])
ax.plot(lag_data,t_height[:h1],color='red',label='Theta',linewidth=3,linestyle='dashed')

#plot CAPE
lag_data,efolding_time = integral_ts(cape,(t_dec - t_dec[0]),t_height[:h1])
ax.plot(lag_data,t_height[:h1],color='blue',label='CAPE',linewidth=3,linestyle='dotted')

#plot wspd
lag_data,efolding_time = integral_ts(wspd,(l_dec - l_dec[0]),l_height[:h2])
ax.plot(sig.medfilt(lag_data,9),l_height[:h2],color='k',label='Wind Speed',linewidth=3,linestyle='dashdot')

#Format axes
ax.set_ylabel('Height [m]')
ax.yaxis.set_major_locator(MultipleLocator(200))
ax.yaxis.set_major_formatter(FormatStrFormatter('%d'))
ax.yaxis.set_minor_locator(MultipleLocator(100))


ax.set_xlabel('Time Scale [hours]')
ax.xaxis.set_major_locator(MultipleLocator(1))
ax.xaxis.set_major_formatter(FormatStrFormatter('%d'))
ax.xaxis.set_minor_locator(MultipleLocator(0.15))
#ax.set_xlim([0,5])
#legend
ax.legend(framealpha=0.0,loc='upper right',fontsize=18)

plt.tight_layout()
#plt.savefig('figures/20170310/time_scales_{}.jpg'.format(datetime.strptime(str(data['date']),'%Y-%m-%d %H:%M:%S').strftime('%Y%m%d')),dpi=300,bbox_inches='tight')
plt.show()
plt.close()